# Despliegue

In [1]:
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
#!pip install openpyxl
import openpyxl

import numpy as np

El despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
En este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.

In [2]:
#### conectarse a base de datos preprocesada

con=sql.connect('bases de datos RRHH/datos.db')
cur=con.cursor()

In [3]:
#crear dataframe para el modelado(2015)
df = pd.read_sql("""select * from completo
where InfoDate='2016-12-31'
""",con=con)

In [4]:
# Hacer la preparacion de los datos nuevos
df_t= fn.preparar_datos(df)

In [5]:
# Cargar modelo entrenado
rf_final = joblib.load("salidas\\rf_final.pkl")

In [6]:
# Realizar predicciones
predicciones = rf_final.predict(df_t)
pd_pred = pd.DataFrame(predicciones, columns=['Atrition'])

# Crear DataFrame con predicciones
perf_pred = pd.concat([df['EmployeeID'], df_t, pd_pred], axis=1)

# Guardar predicciones en archivos
perf_pred[['EmployeeID', 'Atrition']].to_excel("salidas\\prediccion.xlsx")
#Guardar importancia de las caracteristicas a la hora de predecir
feature_names = df_t.columns


In [7]:
importances = pd.DataFrame({'Feature': feature_names, 'Importance': rf_final.feature_importances_})
importances.to_excel("salidas\\importances.xlsx")

In [8]:
# Ver las 10 predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=False).head(15)
emp_pred_bajo.set_index('EmployeeID', inplace=True)
pred = emp_pred_bajo.T
print(pred)

EmployeeID                   7813      6414      6400      6399      6005  \
Age                     -0.758170 -0.210661 -1.743687 -1.853188 -1.196177   
DistanceFromHome        -0.887515 -0.270544 -1.010909 -0.147150  0.593216   
EnvironmentSatisfaction -1.583120  1.169861  0.252200  1.169861  0.252200   
MonthlyIncome           -0.531819 -0.081790 -0.154670 -0.645708  0.213981   
NumCompaniesWorked       0.524943 -1.078797 -0.677862 -0.677862 -1.078797   
PercentSalaryHike       -0.877232  1.582663  0.762698  1.309341  0.762698   
TotalWorkingYears       -0.549605 -0.292524 -1.320847 -1.192307 -0.549605   
YearsAtCompany          -0.327893  0.161947 -0.981014 -0.817734 -0.164613   
Atrition                 1.000000  1.000000  1.000000  1.000000  1.000000   

EmployeeID                   8366      5507      8364      7135      5506  \
Age                     -1.634185  2.088878 -1.743687 -1.196177 -0.320163   
DistanceFromHome        -1.010909 -1.010909 -1.010909 -0.640727  2.197341  

In [10]:
import joblib
import warnings
from sklearn.preprocessing import StandardScaler

# Suprimir advertencias específicas de sklearn
warnings.filterwarnings("ignore", category=UserWarning)

# Cargar el scaler previamente guardado
scaler = joblib.load('salidas/scaler.pkl')

# Seleccionar las columnas escaladas
columnas_escaladas = ['Age', 'EnvironmentSatisfaction', 'MonthlyIncome', 'NumCompaniesWorked', 'TotalWorkingYears', 'YearsAtCompany']

# Extraer las posiciones de las columnas en el scaler original
columnas_idx = [list(scaler.feature_names_in_).index(col) for col in columnas_escaladas]

# Crear un nuevo scaler solo para las columnas seleccionadas
scaler_reducido = StandardScaler()
scaler_reducido.mean_ = scaler.mean_[columnas_idx]
scaler_reducido.scale_ = scaler.scale_[columnas_idx]
scaler_reducido.var_ = scaler.var_[columnas_idx]  # Incluyendo la varianza si es necesario

# Ordenar las predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=False).head(15)
emp_pred_bajo.set_index('EmployeeID', inplace=True)

# Seleccionar solo las columnas que fueron escaladas
df_pred_bajo_escalado = emp_pred_bajo[columnas_escaladas]

# Desescalar las predicciones
df_pred_bajo_original = pd.DataFrame(scaler_reducido.inverse_transform(df_pred_bajo_escalado), columns=df_pred_bajo_escalado.columns, index=df_pred_bajo_escalado.index)

# Concatenar las predicciones desescaladas con las columnas relevantes
df_pred_final = pd.concat([emp_pred_bajo[['Atrition']], df_pred_bajo_original], axis=1)

# Trasponer la tabla final
df_pred_final_traspuesta = df_pred_final.T

# Mostrar los resultados traspuestos
print(df_pred_final_traspuesta)


EmployeeID                  7813     6414     6400     6399     6005     8366  \
Atrition                     1.0      1.0      1.0      1.0      1.0      1.0   
Age                         30.0     35.0     21.0     20.0     26.0     22.0   
EnvironmentSatisfaction      1.0      4.0      3.0      4.0      3.0      3.0   
MonthlyIncome            40000.0  61180.0  57750.0  34640.0  75100.0  49680.0   
NumCompaniesWorked           4.0      0.0      1.0      1.0      0.0      1.0   
TotalWorkingYears            7.0      9.0      1.0      2.0      7.0      1.0   
YearsAtCompany               5.0      8.0      1.0      2.0      6.0      1.0   

EmployeeID                  5507     8364     7135     5506     7459     4966  \
Atrition                     1.0      1.0      1.0      1.0      1.0      1.0   
Age                         56.0     21.0     26.0     34.0     41.0     39.0   
EnvironmentSatisfaction      1.0      2.0      1.0      4.0      1.0      1.0   
MonthlyIncome            25

* Edad: Los empleados predichos a renunciar presentan edades muy diversas, desde 20 hasta 56 años
Satisfacción con el ambiente laboral: La satisfacción laboral de los empleados varía entre valores bajos y moderados. El EmployeeID 7813, 5507, 7135 y 7906 tiene una satisfacción laboral de 1, lo que sugiere una insatisfacción significativa.
Otros empleados, tienen una satisfacción laboral de 3 o 4, lo que indica una satisfacción alta.

* Años en la empresa: Hay una diferencia considerable en los años trabajados en la empresa, desde 1 hasta 18 años. Los empleados que han estado en la empresa durante más tiempo, también están en riesgo de renuncia, lo que podría deberse a factores como el agotamiento laboral o la falta de oportunidades de crecimiento.


Recomendaciones:
* Intervenir en la satisfacción laboral: Dado que varios de los empleados con riesgo de renuncia tienen una baja satisfacción con el ambiente laboral, se recomienda implementar programas para mejorar este aspecto, como fomentar una cultura laboral positiva y ofrecer canales de retroalimentación abierta.

* Revisión salarial: La variabilidad en los ingresos sugiere que podría haber una correlación entre los salarios más bajos y el riesgo de renuncia. Sería recomendable realizar una revisión salarial, especialmente para los empleados con ingresos bajos, para reducir el riesgo de pérdida de talento.

* Desarrollo y oportunidades de crecimiento: Los empleados que han trabajado en la empresa durante muchos años podrían estar enfrentando estancamiento profesional. Ofrecer oportunidades de desarrollo profesional y planes de carrera podría ayudar a retener a estos empleados.

* Monitorear a los empleados jóvenes y con movilidad laboral previa: Los empleados más jóvenes y aquellos que han trabajado en varias empresas parecen tener un mayor riesgo de rotación. Sería prudente ofrecerles más atención para aumentar su compromiso con la empresa.